In [15]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit.primitives import Estimator

from squlearn.feature_map import ChebPQC
from squlearn.kernel.matrix import ProjectedQuantumKernel, FidelityKernel
from squlearn.kernel.ml import QSVR
from squlearn.util import Executor


In [16]:
# Generate data set (analogously to qgpr_workflow.ipynb example)
def func(x):
    return np.sin(3*x)*x*x

# define some noise that can be added to training data
noise_std = 0.1
x = np.linspace(-1, 1, 100)
y = func(x) 
x_train = np.linspace(-1, 1, 30)
y_train = func(x_train) + np.random.normal(scale=noise_std, size=len(x_train))

In [17]:
# We can define an qml method of choice and use additional sklearn functions
# to validate the model

# 1. Step: define QML method
# set up feature map with trainable parameters being randomly initialized
fmap = ChebPQC(
    num_qubits=4, num_features=1, num_layers=2
)
q_kernel_fqk = FidelityKernel(feature_map=fmap, executor=Executor('statevector_simulator'))
np.random.seed(0)
param = np.random.rand(fmap.num_parameters)
q_kernel_fqk.assign_parameters(param)
qsvr_fqk = QSVR(quantum_kernel=q_kernel_fqk, C=1e-3)

In [18]:
# 2. Step load and use additional sklearn routines
from sklearn.model_selection import cross_val_score, cross_val_predict

print("Cross validation score:", cross_val_score(qsvr_fqk, x_train.reshape(-1, 1), y_train, cv=5))
print("Cross validation prediction:", cross_val_predict(qsvr_fqk, x_train.reshape(-1, 1), y_train, cv=5))

Cross validation score: [-45.33618867  -2.76908199  -0.12569225  -1.85667525  -5.68701254]
Cross validation prediction: [ 0.06770502  0.06615496  0.06545925  0.06497734  0.06464847  0.06444862
  0.0548306   0.05522995  0.05568914  0.05619993  0.05675486  0.05734693
 -0.02488046 -0.02408566 -0.02328057 -0.02247447 -0.0216767  -0.02089682
 -0.0567504  -0.05624452 -0.05578105 -0.05536728 -0.05501125 -0.05472222
 -0.08303216 -0.0833036  -0.08367936 -0.08417781 -0.08484747 -0.08622441]


In [21]:
# we could also use sklearn routines to optimize hyperparameters
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1e-3, 1e-2, 1e-1, 1, 10], 'epsilon': [0.01, 0.1, 0.5]}
qsvr_grid = GridSearchCV(qsvr_fqk, param_grid)

In [22]:
qsvr_grid.fit(x_train.reshape(-1, 1), y_train)

GridSearchCV(estimator=QSVR(C=0.001, cache_size=200, epsilon=0.1, max_iter=-1,
                            quantum_kernel=<src.squlearn.kernel.matrix.fidelity_kernel.FidelityKernel object at 0x000001C39DF73790>,
                            shrinking=True, tol=0.001, verbose=False),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10],
                         'epsilon': [0.01, 0.1, 0.5]})

In [23]:
from sklearn.metrics import mean_squared_error as mse
print("Best parameters:", qsvr_grid.best_params_)
print("MSE with optimized hyperparameters:", mse(y_train, qsvr_grid.predict(x_train.reshape(-1, 1))))


Best parameters: {'C': 10, 'epsilon': 0.1}
MSE with optimized hyperparameters: 0.009727059321197988
